# FDS Challenge Notebook

## 0. Version Summary

This version approaches the problem by focusing on move level features, specifically by calculating the expected damage of each move and tracking the available status conditions for the Pokèmon. It also includes cumulative features, such as mean and variance, to predict imbalances between the teams.

This notebook includes evaluation of single models, which we then decided to stack using the StackingClassifier, obtaining better scores than single models. The approach focuses on gradient-boost like algorithms, with LogisticRegression as final estimator. This version also includes a trial of CalibratedClassifier, which we evaluated, given that it could increase metrics via predicted probabilities.

Each model in the StackingClassifier was tuned using HalvingGridSearch, an experimental hyperparameter tuning strategy. This was chosen because it yielded better results than RandomizedSearch, and is significantly faster than GridSearch.

This version evaluates PCA on the dataset before training.

## 1. Loading the Data

In [ ]:
import json
import pandas as pd
import os

# --- Define the path to our data ---
COMPETITION_NAME = 'fds-pokemon-battles-prediction-2025'
DATA_PATH = os.path.join('../input', COMPETITION_NAME)
train_file_path = os.path.join(DATA_PATH, 'train.jsonl')
test_file_path = os.path.join(DATA_PATH, 'test.jsonl')

def load_data(file_path):
    data = []
    print(f"Loading data from '{file_path}'...")
    try:
        with open(file_path, 'r') as f:
            for line in f:
                data.append(json.loads(line))
        print(f"Successfully loaded {len(data)} battles.")
    except FileNotFoundError:
        print(f"ERROR: Could not find the file at '{file_path}'.")
    return data

train_data = load_data(train_file_path)
test_data = load_data(test_file_path) # Load the test data as well

## 2. Complete Pokèmon Dataframe

In [ ]:
import pandas as pd

# Gen 1 types in lowercase
types = [
    "normal", "fire", "water", "electric", "grass", "ice", "fighting", "poison",
    "ground", "flying", "psychic", "bug", "rock", "ghost", "dragon", "notype",
]

# Type effectiveness values
type_chart = {
    "normal":   {"rock": 0.5, "ghost": 0.0},
    "fire":     {"grass": 2.0, "ice": 2.0, "bug": 2.0, "rock": 0.5, "fire": 0.5, "water": 0.5},
    "water":    {"fire": 2.0, "ground": 2.0, "rock": 2.0, "water": 0.5, "grass": 0.5},
    "electric": {"water": 2.0, "flying": 2.0, "electric": 0.5, "grass": 0.5, "ground": 0.0},
    "grass":    {"water": 2.0, "ground": 2.0, "rock": 2.0, "fire": 0.5, "grass": 0.5, "flying": 0.5, "bug": 0.5},
    "ice":      {"grass": 2.0, "ground": 2.0, "flying": 2.0, "dragon": 2.0, "fire": 0.5, "water": 0.5},
    "fighting": {"normal": 2.0, "rock": 2.0, "ice": 2.0, "bug": 0.5, "psychic": 0.5, "ghost": 0.0},
    "poison":   {"grass": 2.0, "bug": 2.0, "poison": 0.5, "ground": 0.5, "rock": 0.5, "ghost": 0.5},
    "ground":   {"fire": 2.0, "electric": 2.0, "poison": 2.0, "rock": 2.0, "bug": 0.5, "flying": 0.0},
    "flying":   {"grass": 2.0, "fighting": 2.0, "bug": 2.0, "electric": 0.5, "rock": 0.5},
    "psychic":  {"fighting": 2.0, "poison": 2.0, "psychic": 0.5},
    "bug":      {"grass": 2.0, "poison": 2.0, "psychic": 2.0, "fire": 0.5, "fighting": 0.5, "flying": 0.5, "ghost": 0.5},
    "rock":     {"fire": 2.0, "ice": 2.0, "flying": 2.0, "bug": 2.0, "fighting": 0.5, "ground": 0.5},
    "ghost":    {"psychic": 0.0, "ghost": 2.0, "normal": 0.0},
    "dragon":   {"dragon": 2.0},
    "notype":   {}
}

# Create full chart with default 1.0 (neutral)
df_typechart = pd.DataFrame(index=types, columns=types).fillna(1.0)

# Apply effectiveness values
for attacker, defenders in type_chart.items():
    for defender, value in defenders.items():
        df_typechart.loc[attacker, defender] = value

In [ ]:
print(df_typechart.head())

In [ ]:
import json
import pandas as pd

def extract_unique_pokemon_no_ids(jsonl_path: str) -> pd.DataFrame:
    """
    Extracts a clean list of unique Pokémon with full base stats from the dataset.
    - Includes Pokémon from p1_team_details, p2_lead_details, and p2_pokemon_state.
    - Removes rows with all-zero stats if the Pokémon appears elsewhere with valid stats.
    - Removes duplicates across battles: only one row per Pokémon name.
    - Drops battle_id column.
    """
    rows = []

    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            battle = json.loads(line)

            # --- p1 team Pokémon ---
            for p in battle.get("p1_team_details", []):
                rows.append({
                    "name": p.get("name", "unknown"),
                    "base_hp": p.get("base_hp", 0),
                    "base_atk": p.get("base_atk", 0),
                    "base_def": p.get("base_def", 0),
                    "base_spa": p.get("base_spa", 0),
                    "base_spd": p.get("base_spd", 0),
                    "base_spe": p.get("base_spe", 0),
                    "type_1": p.get("types", "notype")[0],
                    "type_2": p.get("types", "notype")[1],
                    "lvl": p.get("level", 0),
                    "hp": (2 * p.get("base_hp", 0)) + 100 + 10,
                    "atk": (2 * p.get("base_atk", 0)) + 5,
                    "def": (2 * p.get("base_def", 0)) + 5,
                    "spa": (2 * p.get("base_spa", 0)) + 5,
                    "spd": (2 * p.get("base_spd", 0)) + 5,
                    "spe": (2 * p.get("base_spe", 0)) + 5,
                })

            # --- p2 lead details ---
            lead_details = battle.get("p2_lead_details")
            if lead_details:
                rows.append({
                    "name": lead_details.get("name", "unknown"),
                    "base_hp": lead_details.get("base_hp", 0),
                    "base_atk": lead_details.get("base_atk", 0),
                    "base_def": lead_details.get("base_def", 0),
                    "base_spa": lead_details.get("base_spa", 0),
                    "base_spd": lead_details.get("base_spd", 0),
                    "base_spe": lead_details.get("base_spe", 0),
                    "type_1": lead_details.get("types", "notype")[0],
                    "type_2": lead_details.get("types", "notype")[1],
                    "lvl": lead_details.get("level", 0),

                    # Full stats at level 100 with no IVs/EVs (https://www.pokemaniablog.com/2017/11/11/CalculatingHP.html)
                    "hp" : (2 * lead_details.get("base_hp", 0)) + 100 + 10,
                    "atk": (2 * lead_details.get("base_atk", 0)) + 5,
                    "def": (2 * lead_details.get("base_def", 0)) + 5,
                    "spa": (2 * lead_details.get("base_spa", 0)) + 5,
                    "spd": (2 * lead_details.get("base_spd", 0)) + 5,
                    "spe": (2 * lead_details.get("base_spe", 0)) + 5,
                })

            # --- p2 team Pokémon from timeline (unique per battle) ---
            seen = set()
            for turn in battle.get("battle_timeline", []):
                p2 = turn.get("p2_pokemon_state")
                if not p2:
                    continue
                name = p2.get("name", "unknown")
                if name in seen:
                    continue
                seen.add(name)
                rows.append({
                    "name": name,
                    "base_hp": p2.get("base_hp", 0),
                    "base_atk": p2.get("base_atk", 0),
                    "base_def": p2.get("base_def", 0),
                    "base_spa": p2.get("base_spa", 0),
                    "base_spd": p2.get("base_spd", 0),
                    "base_spe": p2.get("base_spe", 0),
                    "type_1": p2.get("types", "notype")[0],
                    "type_2": p2.get("types", "notype")[1],
                    "lvl": p2.get("level", 0),

                    # Full stats at level 100 with no IVs/EVs (https://www.pokemaniablog.com/2017/11/11/CalculatingHP.html)
                    "hp": (2 * p2.get("base_hp", 0)) + 100 + 10,
                    "atk": (2 * p2.get("base_atk", 0)) + 5,
                    "def": (2 * p2.get("base_def", 0)) + 5,
                    "spa": (2 * p2.get("base_spa", 0)) + 5,
                    "spd": (2 * p2.get("base_spd", 0)) + 5,
                    "spe": (2 * p2.get("base_spe", 0)) + 5,
                })

    df = pd.DataFrame(rows)

    # --- Remove zero-stat rows if name appears elsewhere with valid stats ---
    stat_cols = ["base_hp", "base_atk", "base_def", "base_spa", "base_spd", "base_spe", "lvl"]
    zero_mask = (df[stat_cols] == 0).all(axis=1)
    valid_names = set(df.loc[~zero_mask, "name"])
    df = df.loc[~(zero_mask & df["name"].isin(valid_names))]

    # --- Drop duplicates: keep only one row per Pokémon name ---
    df = df.drop_duplicates(subset=["name"], keep="first").reset_index(drop=True)
    
    def normalize_levels(df):
        def flatten_levels(x):
            if isinstance(x, tuple):
                return list(x)
            elif isinstance(x, list):
                return x
            else:
                return [x]  # single int
    
        df = df.copy()
        df["lvl"] = df["lvl"].apply(flatten_levels)
        return df.explode("lvl").astype({"lvl": int})

    clean_df = normalize_levels(df)

    merged_df = (
        clean_df.groupby("name", as_index=False)
        .agg({
            "base_hp": "max",
            "base_atk": "max",
            "base_def": "max",
            "base_spa": "max",
            "base_spd": "max",
            "base_spe": "max",
            "type_1": "first",
            "type_2": "first",
            "lvl": "max", # Keep the highest level if multiple levels exist
            "hp": "max",
            "atk": "max",
            "def": "max",
            "spa": "max",
            "spd": "max",
            "spe": "max",
        })
    )

    return merged_df


In [ ]:
pokemon_df_train = extract_unique_pokemon_no_ids(train_file_path)

In [ ]:
print("\n All Pokémon entries:")
display(pokemon_df_train)

In [ ]:
def extract_triggered_statuses(jsonl_path: str) -> pd.DataFrame:
    """
    Extracts status conditions triggered during battles from the dataset.
    - For each turn, checks the 'status' field of p1 and p2 pokemon_state.
    - Records battle_id, turn number, player (p1 or p2), and status condition.
    """
    data = set()

    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            battle = json.loads(line)
            timeline = battle.get("battle_timeline", [])

            for turn in timeline:
                for player_key in ["p1", "p2"]:
                    pokemon_state = turn.get(f"{player_key}_pokemon_state", {})
                    status = pokemon_state.get("status")

                    if status != 'nostatus' and status is not None:
                        data.add(status)

    return list(data)


In [ ]:
available_status = extract_triggered_statuses(train_file_path)
print("\n Extracted Status Conditions:")
print(available_status)

In [ ]:
import math


def make_moves_df(jsonl_path: str, pokemon_df: pd.DataFrame, typechart: pd.DataFrame, verbose: bool) -> pd.DataFrame:
    """
    Processes battle data to create a DataFrame of moves with calculated features.
    - For each turn in each battle, extracts move details for both players.
    - Calculates expected damage based on move properties and Pokémon stats.
    - Tracks other features such as type effectiveness, status conditions, and stat boosts.
    """

    import json
    import pandas as pd

    move_rows = []

    #Boosts and relative multipliers available for atk, def, spa, spd, spe
    boost_multipliers = {
        -6: 0.25,
        -5: 0.28,
        -4: 0.33,
        -3: 0.4,
        -2: 0.5,
        -1: 0.66,
        0: 1.0,
        1: 1.5,
        2: 2.0,
        3: 2.5,
        4: 3.0,
        5: 3.5,
        6: 4.0,
    }

    # Normalize Pokémon data
    pokemon_df_copy = pokemon_df.copy()
    pokemon_df_copy.columns = pokemon_df_copy.columns.str.lower().str.strip()

    if "name" in pokemon_df_copy.columns:
        pokemon_df_copy["name"] = pokemon_df_copy["name"].str.lower().str.strip()
        pokemon_df_copy.set_index("name", inplace=True)
    else:
        raise ValueError(f"'name' column missing. Available columns: {pokemon_df_copy.columns.tolist()}")

    # Parse battles
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            battle = json.loads(line)
            battle_id = battle.get("battle_id")
            timeline = battle.get("battle_timeline", [])

            for turn_index, turn in enumerate(timeline):
                turn_data = {}
                turn_moves = []

                for side in ["p1", "p2"]:
                    opponent = "p2" if side == "p1" else "p1"
                    move = turn.get(f"{side}_move_details")
                    if not move:
                        continue

                    atk_boosts = turn.get(f"{side}_pokemon_state").get("boosts", {
                    "atk": 0,
                    "def": 0,
                    "spa": 0,
                    "spd": 0,
                    "spe": 0
                })

                    def_boosts = turn.get(f"{opponent}_pokemon_state").get("boosts", {
                    "atk": 0,
                    "def": 0,
                    "spa": 0,
                    "spd": 0,
                    "spe": 0
                })

                    atk_pk_name = turn.get(f"{side}_pokemon_state").get("name").lower().strip()
                    def_pk_name = turn.get(f"{opponent}_pokemon_state").get("name").lower().strip()

                    atk_spe = pokemon_df_copy.loc[atk_pk_name, "spe"]
                    def_spe = pokemon_df_copy.loc[def_pk_name, "spe"]

                    def_hp_pct = turn.get(f"{opponent}_pokemon_state").get("hp_pct")

                    def_t1 = pokemon_df_copy.loc[def_pk_name, "type_1"]
                    def_t2 = pokemon_df_copy.loc[def_pk_name, "type_2"]
                    def_stat = pokemon_df_copy.loc[def_pk_name, "def"] if move.get("category").lower() == "physical" else pokemon_df_copy.loc[def_pk_name, "spd"]

                    atk_t1 = pokemon_df_copy.loc[atk_pk_name, "type_1"]
                    atk_t2 = pokemon_df_copy.loc[atk_pk_name, "type_2"]
                    atk_stat = pokemon_df_copy.loc[atk_pk_name, "atk"] if move.get("category").lower() == "physical" else pokemon_df_copy.loc[atk_pk_name, "spa"]

                    
                    if move.get("category").lower() == "physical":
                        def_stat = def_stat * boost_multipliers.get(def_boosts.get("def"), 1.0)
                        atk_stat = atk_stat * boost_multipliers.get(atk_boosts.get("atk"), 1.0)
                    elif move.get("category").lower() == "special":
                        def_stat = def_stat * boost_multipliers.get(def_boosts.get("spd"), 1.0)
                        atk_stat = atk_stat * boost_multipliers.get(atk_boosts.get("spa"), 1.0)

                    atk_spe = atk_spe * boost_multipliers.get(atk_boosts.get("spe"), 1.0)
                    def_spe = def_spe * boost_multipliers.get(def_boosts.get("spe"), 1.0)

                    if move.get("category").lower() == "physical" and move.get("name").lower() == "reflect":
                        def_stat *= 2
                    elif move.get("category").lower() == "special" and move.get("name").lower() == "lightscreen":
                        def_stat *= 2

                    if move.get("name").lower() in ["explosion", "selfdestruct"]:
                        def_stat = max(1, def_stat // 2)

                    if atk_stat > 255 or def_stat > 255:
                        atk_stat = math.floor(atk_stat / 4)
                        def_stat = max(1, math.floor(def_stat / 4))
                    
                    move_mul = (typechart.loc[move.get("type").lower(), def_t1] * 
                                typechart.loc[move.get("type").lower(), def_t2]) if move.get("category").lower() != "status" else 1.0

                    turn_data[side] = {
                        "priority": move.get("priority", 0),
                        "speed": atk_spe
                    }

                    turn_moves.append({
                        "battle_id": battle_id,
                        "turn": turn_index,
                        "attacker": side,
                        "atk_pk": atk_pk_name,
                        "atk_t1": atk_t1,
                        "atk_t2": atk_t2,
                        "name": move.get("name"),
                        "move_type": move.get("type").lower(),
                        "category": move.get("category").lower(),
                        "base_power": move.get("base_power"),
                        "accuracy": move.get("accuracy"),
                        "priority": move.get("priority"),
                        "defender": opponent,
                        "def_pk": def_pk_name,
                        "def_t1": def_t1,
                        "def_t2": def_t2,
                        "stab": 1 if move.get("type").lower() in [atk_t1, atk_t2] else 0,

                        # Simplified damage formula from Bulbapedia (https://bulbapedia.bulbagarden.net/wiki/Damage#Damage_calculation)
                        "tot_damage": 0.0 if move.get("category").lower() == "status" else 
                                    ((((42 * move.get("base_power", 0) * atk_stat / def_stat) / 50) + 2) *
                                      (1.5 if move.get("type").lower() in [atk_t1, atk_t2] else 1.0) *
                                      move_mul * 0.925),  # average random factor 0.85-1.0
                        "se_move": 1 if move_mul == 2.0 else 0,
                        "pe_move": 1 if move_mul == 0.5 else 0,
                        "ne_move": 1 if move_mul == 0.0 else 0,

                        "ko": 1 if def_hp_pct == 0.0 else 0,
                        "def_status": turn.get(f"{opponent}_pokemon_state").get("status"),
                        "def_fnt": 1 if turn.get(f"{opponent}_pokemon_state").get("status") == "fnt" else 0,
                        "def_par": 1 if turn.get(f"{opponent}_pokemon_state").get("status") == "par" else 0,
                        "def_slp": 1 if turn.get(f"{opponent}_pokemon_state").get("status") == "slp" else 0,
                        "def_frz": 1 if turn.get(f"{opponent}_pokemon_state").get("status") == "frz" else 0,
                        "def_brn": 1 if turn.get(f"{opponent}_pokemon_state").get("status") == "brn" else 0,
                        "def_psn": 1 if turn.get(f"{opponent}_pokemon_state").get("status") == "psn" else 0,
                        "atk_status": turn.get(f"{side}_pokemon_state").get("status"),
                        "atk_fnt": 1 if turn.get(f"{side}_pokemon_state").get("status") == "fnt" else 0,
                        "atk_par": 1 if turn.get(f"{side}_pokemon_state").get("status") == "par" else 0,
                        "atk_slp": 1 if turn.get(f"{side}_pokemon_state").get("status") == "slp" else 0,
                        "atk_frz": 1 if turn.get(f"{side}_pokemon_state").get("status") == "frz" else 0,
                        "atk_brn": 1 if turn.get(f"{side}_pokemon_state").get("status") == "brn" else 0,
                        "atk_psn": 1 if turn.get(f"{side}_pokemon_state").get("status") == "psn" else 0,
                        "atk_advantage": 1 if ((
                            ((typechart.loc[atk_t1, def_t1] * typechart.loc[atk_t1, def_t2] >= 2.0) or (typechart.loc[atk_t2, def_t1] * typechart.loc[atk_t2, def_t2] >= 2.0)) or
                            ((0.0 <= typechart.loc[def_t1, atk_t1] * typechart.loc[def_t1, atk_t2] <= 0.5) or 
                             (0.0 <= typechart.loc[def_t2, atk_t1] * typechart.loc[def_t2, atk_t2] <= 0.5))) and move.get("category").lower() != "status"
                        ) else 0,
                        f"status_changed_{side}": 1 if turn.get(f"{side}_pokemon_state").get("status") != "nostatus" else 0,
                        f"status_changed_{opponent}": 1 if turn.get(f"{opponent}_pokemon_state").get("status") != "nostatus" else 0,
                        f"effects_{side}": turn.get(f"{side}_pokemon_state").get("effects"),
                        f"effects_{opponent}": turn.get(f"{opponent}_pokemon_state").get("effects"),
                        f"{side}_effect_changed": 1 if turn.get(f"{side}_pokemon_state").get("effects") != ["noeffect"] else 0,
                        f"{opponent}_effect_changed": 1 if turn.get(f"{opponent}_pokemon_state").get("effects") != ["noeffect"] else 0,
                        f"boosted_{side}_atk": atk_boosts.get("atk", 0),
                        f"boosted_{side}_def": atk_boosts.get("def", 0),
                        f"boosted_{side}_spa": atk_boosts.get("spa", 0),
                        f"boosted_{side}_spd": atk_boosts.get("spd", 0),
                        f"boosted_{side}_spe": atk_boosts.get("spe", 0),
                        f"boosted_{opponent}_atk": def_boosts.get("atk", 0),
                        f"boosted_{opponent}_def": def_boosts.get("def", 0),
                        f"boosted_{opponent}_spa": def_boosts.get("spa", 0),
                        f"boosted_{opponent}_spd": def_boosts.get("spd", 0),
                        f"boosted_{opponent}_spe": def_boosts.get("spe", 0),
                        "atk_hp_pct": turn.get(f"{side}_pokemon_state").get("hp_pct"),
                        "def_hp_pct": turn.get(f"{opponent}_pokemon_state").get("hp_pct"),
                    })                            

                # Decide who attacks first
                p1 = turn_data.get("p1", {"priority": 0, "speed": 0})
                p2 = turn_data.get("p2", {"priority": 0, "speed": 0})

                if p1["priority"] > p2["priority"]:
                    first = "p1"
                elif p2["priority"] > p1["priority"]:
                    first = "p2"
                else:
                    first = "p1" if p1["speed"] > p2["speed"] else "p2" if p2["speed"] > p1["speed"] else "tie"

                # Assign first attacker to all moves in this turn
                for row in turn_moves:
                    row["first_attacker"] = first
                    move_rows.append(row)

    moves_df = pd.DataFrame(move_rows)
    moves_df["name"] = moves_df["name"].str.lower().str.strip()

    # --- Check for duplicates and NaN values ---

    if verbose:
        print("Checking for duplicates and NaN values...")

        if moves_df.columns.duplicated().any():
            print("Duplicate columns found:")
            print(moves_df[moves_df.columns[moves_df.columns.duplicated(keep=False)]])

        if moves_df.isnull().values.any():
            print("NaN values found:")
            print(moves_df[moves_df.isnull().any(axis=1)])

    return moves_df


In [ ]:
moves_df_train = make_moves_df(train_file_path, pokemon_df_train, df_typechart, verbose=True)

In [ ]:
display(moves_df_train[[def_col for def_col in moves_df_train.columns if def_col.startswith("atk_")]].head())

## 3. Feature engineering (finally)

In [ ]:
import json
import pandas as pd

def compute_full_features(moves_df: pd.DataFrame, pokemon_df: pd.DataFrame, jsonl_path: str, verbose: bool) -> pd.DataFrame:
    # --- Normalize Pokémon data ---
    pokemon_df = pokemon_df.copy()
    pokemon_df.columns = pokemon_df.columns.str.lower().str.strip()
    pokemon_df["name"] = pokemon_df["name"].str.lower().str.strip()
    pokemon_df.set_index("name", inplace=True)

    stat_cols = ["hp", "atk", "def", "spa", "spd", "spe"]

    # --- Clean moves_df ---
    moves_df = moves_df.copy()
    moves_df["player"] = moves_df["attacker"]
    moves_df["atk_pk"] = moves_df["atk_pk"].str.lower().str.strip()

    # --- Move category counts ---
    category_counts = (
        moves_df.groupby(["battle_id", "player", "category"])
        .size()
        .unstack(fill_value=0)
        .rename(columns={
            "status": "num_status_moves",
            "physical": "num_physical_moves",
            "special": "num_special_moves"
        })
        .reset_index()
    )

    # --- Status changes per player ---
    status_changes_p1 = (
        moves_df[moves_df["player"] == "p1"]
        .groupby(["battle_id", "player"])["status_changed_p1"]
        .sum()
        .reset_index()
        .rename(columns={"status_changed_p1": "num_status_changes"})
    )

    status_changes_p2 = (
        moves_df[moves_df["player"] == "p2"]
        .groupby(["battle_id", "player"])["status_changed_p2"]
        .sum()
        .reset_index()
        .rename(columns={"status_changed_p2": "num_status_changes"})
    )

    status_changes = pd.concat([status_changes_p1, status_changes_p2], ignore_index=True)

    # --- Count how many frz, par, slp, brn, psn status changes per player ---
    status_types = ["frz", "par", "slp", "brn", "psn", "fnt"]
    status_type_counts = []
    for status in status_types:
        status_count = (
            moves_df[moves_df["player"] == "p1"]
            .groupby(["battle_id", "player"])[f"def_{status}"]
            .sum()
            .reset_index()
            .rename(columns={f"def_{status}": f"num_{status}_inflicted"})
        )
        status_type_counts.append(status_count)

        status_count = (
            moves_df[moves_df["player"] == "p2"]
            .groupby(["battle_id", "player"])[f"def_{status}"]
            .sum()
            .reset_index()
            .rename(columns={f"def_{status}": f"num_{status}_inflicted"})
        )
        status_type_counts.append(status_count)

    status_type_counts_df = pd.concat(status_type_counts, ignore_index=True)

    # --- Boosts used per player per feature ---
    for stat in ["atk", "def", "spa", "spd", "spe"]:
        moves_df[f"boosted_{stat}"] = moves_df.apply(
            lambda row: row[f"boosted_p1_{stat}"] if row["player"] == "p1" else row[f"boosted_p2_{stat}"],
            axis=1
        )
    
    boosts_summary = (
        moves_df.groupby(["battle_id", "player"])[[f"boosted_{stat}" for stat in ["atk", "def", "spa", "spd", "spe"]]]
        .sum()
        .reset_index()
        .rename(columns={f"boosted_{stat}": f"total_boosts_{stat}" for stat in ["atk", "def", "spa", "spd", "spe"]})
    )

    # --- Accuracy ---
    acc_df = (
        moves_df.groupby(["battle_id", "player"])["accuracy"]
        .mean()
        .reset_index()
        .rename(columns={"accuracy": "avg_accuracy"})
    )

    # --- Which player had the most priority moves ---
    priority_df = (
        moves_df.groupby(["battle_id", "player"])["priority"]
        .sum()
        .reset_index()
        .rename(columns={"priority": "total_priority"})
    )

    # --- Number of moves with accuracy = 1.0 for each player ---
    accurate_moves_df = moves_df[moves_df["accuracy"] == 1.0]
    hits = (
        accurate_moves_df.groupby(["battle_id", "player"])
        .size()
        .reset_index()
        .rename(columns={0: "num_accurate_moves"})
    )

    # --- Number of moves with accuracy < 1.0 for each player ---
    inaccurate_moves_df = moves_df[moves_df["accuracy"] < 1.0]
    misses = (
        inaccurate_moves_df.groupby(["battle_id", "player"])
        .size()
        .reset_index()
        .rename(columns={0: "num_inaccurate_moves"})
    )

    # --- Merge Pokémon stats ---
    stat_merge = moves_df[["battle_id", "player", "atk_pk"]]
    stat_merge = stat_merge.merge(pokemon_df[stat_cols], left_on="atk_pk", right_index=True, how="left")
    stat_merge[stat_cols] = stat_merge[stat_cols].fillna(0)

    # --- Mean stats per player ---
    mean_stats_df = (
        stat_merge.groupby(["battle_id", "player"])[stat_cols]
        .mean()
        .reset_index()
        .rename(columns={col: f"mean_{col}" for col in stat_cols})
    )

    # --- Variance on stats per player ---
    var_stats_df = (
        stat_merge.groupby(["battle_id", "player"])[stat_cols]
        .var()
        .reset_index()
        .rename(columns={col: f"var_{col}" for col in stat_cols})
    )

    # --- Sum stats per player ---
    sum_stats_df = (
        stat_merge.groupby(["battle_id", "player"])[stat_cols]
        .sum()
        .reset_index()
        .rename(columns={col: f"sum_{col}" for col in stat_cols})
    )

    # --- Sum Damage per player ---
    damage_df = (
        moves_df.groupby(["battle_id", "player"])["tot_damage"]
        .sum()
        .reset_index()
        .rename(columns={"tot_damage": "total_damage"})
    )

    # --- Mean Damage per player ---
    mean_damage_df = (
        moves_df.groupby(["battle_id", "player"])["tot_damage"]
        .mean()
        .reset_index()
        .rename(columns={"tot_damage": "mean_damage"})
    )

    # --- Effectiveness counts ---
    eff_df = moves_df.copy()
    eff_df["super_effective"] = eff_df["se_move"]
    eff_df["not_effective"] = eff_df["pe_move"]
    eff_df["neutral"] = 1 - (eff_df["se_move"] + eff_df["pe_move"] + eff_df["ne_move"])

    eff_counts = (
        eff_df.groupby(["battle_id", "player"])[["super_effective", "neutral", "not_effective"]]
        .sum()
        .reset_index()
        .rename(columns={
            "super_effective": "num_super_effective",
            "neutral": "num_neutral",
            "not_effective": "num_not_effective"
        })
    )

    # --- Total pokemon switches per player ---
    moves_df = moves_df.sort_values(by=["battle_id", "turn"])
    moves_df["prev_atk_pk"] = moves_df.groupby(["battle_id", "player"])["atk_pk"].shift(1)
    
    moves_df["is_switch"] = (moves_df["atk_pk"] != moves_df["prev_atk_pk"]) & (moves_df["prev_atk_pk"].notna())
    switches_df = (
        moves_df.groupby(["battle_id", "player"])["is_switch"]
        .sum()
        .reset_index()
        .rename(columns={"is_switch": "num_switches"})
    )

    # --- Estimate drops hp_pct (actual hits) ---
    moves_df["prev_def_hp_pct"] = moves_df.groupby(["battle_id", "defender"])["def_hp_pct"].shift(1)
    moves_df["has_been_hit"] = (
        (moves_df["def_hp_pct"] < moves_df["prev_def_hp_pct"]) &
        (moves_df["prev_def_hp_pct"].notna())
    )

    actual_damage_df = (
        moves_df.groupby(["battle_id", "defender"])["has_been_hit"]
        .sum()
        .reset_index()
        .rename(columns={"defender": "player", "has_been_hit": "num_times_hit"})
    )

    # --- Pokemon has regenerated HP (e.g., via Recover) if pokemon has not been switched ---
    moves_df["prev_atk_hp_pct"] = moves_df.groupby(["battle_id", "player"])["atk_hp_pct"].shift(1)
    moves_df["has_regenerated"] = (
        (moves_df["atk_hp_pct"] > moves_df["prev_atk_hp_pct"]) &
        (moves_df["prev_atk_hp_pct"].notna()) &
        (~moves_df["is_switch"])
    )

    regen_df = (
        moves_df.groupby(["battle_id", "player"])["has_regenerated"]
        .sum()
        .reset_index()
        .rename(columns={"has_regenerated": "num_regens"})
    )

    # --- Total KO ---
    kill_df = (
        moves_df.groupby(["battle_id", "player"])["ko"]
        .sum()
        .reset_index()
        .rename(columns={"ko": "ko_count"})
    )

    # --- Type advantage moves ---
    adv_df = (
        moves_df.groupby(["battle_id", "player"])["atk_advantage"]
        .sum()
        .reset_index()
        .rename(columns={"atk_advantage": "num_advantage_moves"})
    )

    # --- Count occurrences where player attacks first ---
    first_attacker_counts = (
        moves_df[moves_df["first_attacker"] == moves_df["player"]]
        .groupby(["battle_id", "player"])
        .size()
        .reset_index()
        .rename(columns={0: "num_first_attacks"})
    )

    # --- p2 attacks first count ---
    second_attacker_counts = (
        moves_df[moves_df["first_attacker"] != moves_df["player"]]
        .groupby(["battle_id", "player"])
        .size()
        .reset_index()
        .rename(columns={0: "num_second_attacks"})
    )

    # --- Count how many times effects changed per player ---
    moves_df["effect_changed"] = moves_df.apply(
        lambda row: row["p1_effect_changed"] if row["player"] == "p1" else row["p2_effect_changed"],
        axis=1
    )

    effects_summary = (
        moves_df.groupby(["battle_id", "player"])[["effect_changed"]]
        .sum()
        .reset_index()
        .rename(columns={
            "effect_changed": "num_effects_changed"
        })
    )

    # --- Merge all player-level features ---
    player_df = (
        category_counts
        .merge(acc_df, on=["battle_id", "player"], how="outer")
        .merge(mean_stats_df, on=["battle_id", "player"], how="outer")
        .merge(sum_stats_df, on=["battle_id", "player"], how="outer")
        .merge(var_stats_df, on=["battle_id", "player"], how="outer")
        .merge(eff_counts, on=["battle_id", "player"], how="outer")
        .merge(kill_df, on=["battle_id", "player"], how="outer")
        .merge(status_changes, on=["battle_id", "player"], how="outer")
        .merge(status_type_counts_df, on=["battle_id", "player"], how="outer")
        .merge(damage_df, on=["battle_id", "player"], how="outer")
        .merge(boosts_summary, on=["battle_id", "player"], how="outer")
        .merge(priority_df, on=["battle_id", "player"], how="outer")
        .merge(adv_df, on=["battle_id", "player"], how="outer")
        .merge(first_attacker_counts, on=["battle_id", "player"], how="outer")
        .merge(second_attacker_counts, on=["battle_id", "player"], how="outer")
        .merge(hits, on=["battle_id", "player"], how="outer")
        .merge(misses, on=["battle_id", "player"], how="outer")
        .merge(effects_summary, on=["battle_id", "player"], how="outer")
        .merge(switches_df, on=["battle_id", "player"], how="outer")
        .merge(regen_df, on=["battle_id", "player"], how="outer")
        .merge(actual_damage_df, on=["battle_id", "player"], how="outer")
        .fillna(0)
    )

    player_df = (
        player_df
        .groupby(["battle_id", "player"])
        .sum(numeric_only=True)
        .reset_index()
    )


    dupes = player_df.groupby(["battle_id", "player"]).size()
    dupes = dupes[dupes > 1]
    print(dupes)

    # Reset index after pivoting
    battle_df = (
        player_df.pivot(index="battle_id", columns="player")
        .sort_index(axis=1)
        .reset_index()
    )

    # Separate out the pivoted columns
    pivot_cols = battle_df.columns.drop("battle_id")
    battle_df.columns = ["battle_id"] + [f"{player}_{feature}" for feature, player in pivot_cols]

    # --- Add diffs of stats ---
    battle_df["mean_spe_diff"] = battle_df["p1_mean_spe"] - battle_df["p2_mean_spe"]
    battle_df["mean_atk_diff"] = battle_df["p1_mean_atk"] - battle_df["p2_mean_atk"]
    battle_df["mean_def_diff"] = battle_df["p1_mean_def"] - battle_df["p2_mean_def"]
    battle_df["mean_spa_diff"] = battle_df["p1_mean_spa"] - battle_df["p2_mean_spa"]
    battle_df["mean_spd_diff"] = battle_df["p1_mean_spd"] - battle_df["p2_mean_spd"]
    battle_df["mean_hp_diff"] = battle_df["p1_mean_hp"] - battle_df["p2_mean_hp"]

    battle_df["var_atk_diff"] = battle_df["p1_var_atk"] - battle_df["p2_var_atk"]
    battle_df["var_def_diff"] = battle_df["p1_var_def"] - battle_df["p2_var_def"]
    battle_df["var_spa_diff"] = battle_df["p1_var_spa"] - battle_df["p2_var_spa"]
    battle_df["var_spd_diff"] = battle_df["p1_var_spd"] - battle_df["p2_var_spd"]
    battle_df["var_spe_diff"] = battle_df["p1_var_spe"] - battle_df["p2_var_spe"]
    battle_df["var_hp_diff"] = battle_df["p1_var_hp"] - battle_df["p2_var_hp"]

    # --- Add diff switches ---
    battle_df["diff_num_switches"] = battle_df["p1_num_switches"] - battle_df["p2_num_switches"]

    print(battle_df.columns.tolist())

    # --- Add player_won labels ---
    labels = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            battle = json.loads(line)
            labels.append({
                "battle_id": battle.get("battle_id"),
                "player_won": battle.get("player_won", None)
            })
    labels_df = pd.DataFrame(labels)

    # --- Merge winner label into final df ---
    battle_df = battle_df.merge(labels_df, on="battle_id", how="left")

    # SANITY CHECKS
    if verbose:
        print("Performing sanity checks on final DataFrame...")

        # --- Ensure both players exist per battle ---
        all_battles = moves_df["battle_id"].unique()
        expected_players = ["p1", "p2"]
        full_index = pd.MultiIndex.from_product([all_battles, expected_players], names=["battle_id", "player"])
        player_df = player_df.set_index(["battle_id", "player"]).reindex(full_index).reset_index().fillna(0)

        # --- Check that both players are in all battles ---
        expected_battles = set(moves_df["battle_id"].unique())
        actual_battles = set(battle_df["battle_id"].unique())
        missing_battles = expected_battles - actual_battles
        if missing_battles:
            print(f"WARNING: Missing battles in final DataFrame: {missing_battles}")

        # --- Check that there are no duplicate columns ---
        if battle_df.columns.duplicated().any():
            duplicated_cols = battle_df.columns[battle_df.columns.duplicated()].unique()
            print(f"WARNING: Duplicate columns found in final DataFrame: {duplicated_cols.tolist()}")

        # --- Check for NaN values ---
        if battle_df.isnull().values.any():
            print("WARNING: NaN values found in final DataFrame.")
            print(battle_df[battle_df.isnull().any(axis=1)])

        # --- Check duplicate merges ---
        if battle_df.duplicated(subset=["battle_id"]).any():
            print("WARNING: Duplicate battle_id entries found in final DataFrame.")
            print(battle_df[battle_df.duplicated(subset=["battle_id"], keep=False)])

        print("Data is sane (unlike me)")

    return battle_df


In [ ]:
train_df = compute_full_features(moves_df_train, pokemon_df_train, train_file_path, verbose=True)

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 180)
display(train_df.head())

# see dimensionality of final training dataframe
print(f"Training DataFrame shape: {train_df.shape}")


## 4. Training model

In [ ]:
# --- Imports (a lot of them) ---

from sklearn.experimental import enable_halving_search_cv

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, RandomizedSearchCV, GridSearchCV, StratifiedKFold, HalvingGridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from xgboost.sklearn import XGBClassifier
import numpy as np
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
X = train_df.drop(columns=["battle_id", "player_won"])
y = train_df["player_won"]

# Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# --- Training singular models for a different Stacking Classifier later ---

# Define parameter grid
logreg_param_grid = {
    "C": [0.01, 0.1, 1, 10],
    "penalty": ["l2"],
    "solver": ["liblinear", "saga"] # saga does not converge, leaving it for exploration
}

# Model
logreg = LogisticRegression(random_state=42, max_iter=1000)

# Halving Grid Search
logreg_search = HalvingGridSearchCV(
    estimator=logreg,
    param_grid=logreg_param_grid,
    scoring="accuracy",
    cv=5,
    verbose=1,
    n_jobs=-1
)

# Define parameter grid
rf_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["sqrt", "log2"]
}

# Model
rf = RandomForestClassifier(random_state=42)

# Halving Grid Search
rf_search = HalvingGridSearchCV(
    estimator=rf,
    param_grid=rf_param_grid,
    scoring="accuracy",
    cv=5,
    verbose=0,
    n_jobs=-1
)


logreg_search.fit(X_train, y_train)
rf_search.fit(X_train, y_train)

print("Logistic Regression best score:", logreg_search.best_score_)
print("Best params:", logreg_search.best_params_)

print("Random Forest best score:", rf_search.best_score_)
print("Best params:", rf_search.best_params_)


In [ ]:
# Define parameter distributions
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [3, 4, 5],
    "learning_rate": [0.05, 0.1],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "min_child_weight": [1, 3],
    "gamma": [0, 0.1, 0.2],
    "reg_alpha": [0, 0.1, 0.5],
    "reg_lambda": [1, 1.5, 2]
}

# 2. Set up the model
xgb = XGBClassifier(use_label_encoder=False, 
                    eval_metric="logloss", 
                    random_state=42, 
                    objective="binary:logistic"
                    )

# 3. Run Halving Grid Search (normal GridSearchCV takes WAY too long)
random_search = HalvingGridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring="accuracy",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

random_search.fit(X_train, y_train)

# Check train dataset dimensions
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

# 4. Results
print("Best parameters:", random_search.best_params_)
print("Best CV score:", random_search.best_score_)

In [ ]:
best_xgb = random_search.best_estimator_
best_xgb.fit(X_train, y_train)
y_pred = best_xgb.predict(X_val)
print(classification_report(y_val, y_pred))

model=best_xgb

In [ ]:
# --- StackingClassfier with XGB, RandomForest and CatBoost, tuned LogisticRegression as final estimator ---
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier

# --- Define CatBoost model ---
catboost_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    verbose=0,  # suppress training output
    loss_function='Logloss',
    random_seed=42
)

stacked_model2 = StackingClassifier(
    estimators=[
        ("xgb", best_xgb),
        ("rf", rf_search.best_estimator_),
        ("catboost", catboost_model)
    ],
    final_estimator=LogisticRegression(**logreg_search.best_params_),
    cv=5,
    passthrough=True,
    n_jobs=-1
)
stacked_model2.fit(X_train, y_train)

# Evaluate
y_pred = stacked_model2.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_pred))

## Checking feature importance

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

# Calibrate the stacking classifier -> probas can increase metrics
calibrated_stack = CalibratedClassifierCV(estimator=stacked_model2, 
                                          method="sigmoid", 
                                          cv=5,
                                        )
calibrated_stack.fit(X_train, y_train)

# Predict calibrated probabilities
calibrated_probs = calibrated_stack.predict_proba(X_val)[:, 1]

# Tune decision threshold for F1 score
best_thresh = 0.5
best_score = 0

for t in np.linspace(0.3, 0.7, 100):
    preds = (calibrated_probs > t).astype(int)
    score = f1_score(y_val, preds)
    if score > best_score:
        best_score = score
        best_thresh = t

print("Best threshold:", best_thresh)
print("Best F1 score:", best_score)

# Include accuracy at best threshold
final_preds = (calibrated_probs > best_thresh).astype(int)
final_accuracy = accuracy_score(y_val, final_preds)
print("Final Accuracy at best threshold:", final_accuracy)

# ROC AUC score
roc_auc = roc_auc_score(y_val, calibrated_probs)
print("ROC AUC Score:", roc_auc)

final_preds = (calibrated_probs > best_thresh).astype(int)
print("Final Predictions:", final_preds)


In [ ]:
# --- Set up dimensionality reduction with PCA ---
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components="mle")
X_pca = pca.fit_transform(X_scaled)
print(f"Original shape: {X_scaled.shape}, PCA reduced shape: {X_pca.shape}")

In [ ]:
# Train model
X_train_pca, X_val_pca, y_train_pca, y_val_pca = train_test_split(X_pca, y, test_size=0.2, random_state=42, stratify=y)

sm_pca = stacked_model2.fit(X_train_pca, y_train_pca)
cs_pca = calibrated_stack.fit(X_train_pca, y_train_pca)

# Evaluate
preds_sm2 = stacked_model2.predict(X_val_pca)
print("Accuracy (stacked_model2):", accuracy_score(y_val_pca, preds_sm2))

preds_cal_sm2 = calibrated_stack.predict(X_val_pca)
print("Accuracy (calibrated_stack):", accuracy_score(y_val_pca, preds_cal_sm2))

In [ ]:
# diff in accuracy
print("Final accuracy of calibrated_stack before PCA:", final_accuracy)
print("Final accuracy of stacked_model2 before PCA:", accuracy_score(y_val, y_pred))

diff_accuracy = accuracy_score(y_val_pca, preds_cal_sm2) - final_accuracy
diff_accuracy_sm = accuracy_score(y_val_pca, preds_sm2) - accuracy_score(y_val, y_pred)

print("Difference in accuracy after PCA (calibrated_stack):", diff_accuracy)
print("Difference in accuracy after PCA (stacked_model2):", diff_accuracy_sm)

## 5. Creating the Submission File

In [ ]:
# Prepare test features
pokemon_df_test = extract_unique_pokemon_no_ids(test_file_path)
moves_df_test = make_moves_df(test_file_path, pokemon_df_test, df_typechart, verbose=False)
test_df = compute_full_features(moves_df_test, pokemon_df_test, test_file_path, verbose=False)

X_test_raw = test_df.drop(columns=["battle_id", "player_won"])

# Apply same scaling and PCA
X_test_scaled = scaler.transform(X_test_raw)  # use the same scaler as training
X_test_pca = pca.transform(X_test_scaled)     # use the same PCA as training

# Make predictions
print("Generating predictions on the test set...")
test_predictions_cs = calibrated_stack.predict(X_test_pca)

# Create submission DataFrame
submission_df_cs = pd.DataFrame({
    "battle_id": test_df["battle_id"],
    "player_won": test_predictions_cs
})

# Trial with stacked_model2 gave the same public lb score, we chose to select the calibrated_stack model because it was stronger in all trials
# test_predictions_sm = stacked_model2.predict(X_test_pca)
# submission_df_sm = pd.DataFrame({
#     "battle_id": test_df["battle_id"],
#     "player_won": test_predictions_sm
# })

# 5. Save to CSV
submission_df_cs.to_csv("submission_cs.csv", index=False)
# submission_df_sm.to_csv("submission_sm.csv", index=False)

print("\n'submission_cs.csv' file created successfully!")
# print("\n'submission_sm.csv' file created successfully!")